# Pysius - a Python solver for Blausius equation

In [1]:
import numpy as np

Let $f(\eta)$ be the non-dimensional stream function, with $\eta$ being the similarity variable. Blausius equation reads:

$$f''' + \frac{1}{2}ff'' = 0$$

This is a third order ODE, that therefore can be cast to a system of 3 first order ODEs:

$$\begin{cases}
f' = u\\
u'  = \xi\\
\xi' = -\frac{1}{2}f\xi
\end{cases}$$

the first two rows representing de-facto the definitions of the additional variables $f$ and $\xi$. Boundary conditions are:

$$f(0)=0$$
$$u(0)=0$$
$$\lim_{\eta \to \infty}u(\eta)=\int_0^\infty \xi d\eta = 1$$

A complete set of initial conditions is not available (the value of $\xi$ at $\eta=0$ is not provided). This makes it impossible to solve the equation with a time $\eta$-marching scheme (meaning, starting from initial conditions towards higher values of eta), hence they need to be solved nonlinearly.

The theoretical domain would be $\eta\in[0,\infty)$; a discretisation is needed. Here, $\infty$ is approximated with a high enough value `eta_max`, while the parameter `N` indicates on how many nodes the function is evaluated (including extremes, so including $0$ and `eta_max`. `deta` is the spacing between nodes.

_Hint: try starting from a low number of nodes and a low value for_ `eta_max` _and gradually increase them. Do this until no substantial change in the final solution is observed._

In [2]:
eta_max = 20 # extreme of domain
N = 1000 # number of nodes
deta = eta_max / (N-1)

Next up, it has to be decided how to store the unknowns in memory. There are three variables, $f$, $u$ and $\xi$, and $N$ nodes - which adds up to $3N$ unknowns. All variables are stored in a one-dimensional array $y$ of size $3N$:

In [3]:
y = np.zeros(3*N)

It is conventionally decided that the first N elements of `y` represent $\xi$ at each of the nodes, namely:
```python
xi = y[0:N] # where xi[n-1] represents xi at the n-th node
```
notice that the first extreme ($0$) is inclusive, the second ($N$) is not. The other variables just follow:
```python
u = y[N:2*N]
f = y[2*N:3*N]
```

Using a forward-Euler discretisation, the equation can be evaluated at each node $n$ as:

$$
\left(
\begin{bmatrix}
\xi_{n+1}\\
u_{n+1}\\
f_{n+1}
\end{bmatrix} - 
\begin{bmatrix}
\xi_{n}\\
u_{n}\\
f_{n}
\end{bmatrix} 
\right) \frac{1}{\Delta \eta} +
\begin{bmatrix}
\frac{1}{2}\xi_{n}f_n\\
-\xi_{n}\\
-u_{n}
\end{bmatrix} = 0
$$

Notice that such equation cannot be written for $n=N$ (the last cell), as it would require informations from the cell $N+1$ - which does not exist. Therefore, $3(N-1)$ equations can be written; the remaining 3 equations are provided by boundary conditions, so that the number of equations matches the one of unknowns.

This can be written in the current context as:

$$
Ay + b(y) = 0
$$

where $A$ is a matrix, vector $b$ is a function of y; this is written so that:
- the first $N-1$ rows represent the discretised Blausius equation for $\xi$
- the next row represents the boundary contition for $u(0)$
- the next $N-1$ rows represent the discretised Blausius equation for $u$
- the next 2 rows represents the boundary condition for $u(\infty)$
- the next row is the boundary condition for $f$ at $0$
- the final $N-1$ rows represent the discretised Blausius equation for $f$

In [4]:
# Keep in mind that when slicing or using "range", the first bound is inclusive
# while the second is not!
# In this way, for instance, range(1,11) produces an array of numbers from 1 to 10,
# which has size 11-1 = 10.

A = np.zeros((3*N, 3*N))

# Blausius equation for xi (derivative of xi)
for ii in range(N-1):
    A[ii,ii] = -1/deta
    A[ii,ii+1] = 1/deta

# boundary condition for u(0)
A[N-1, N] = 1

# Blausius equation for u (derivative of u)
for ii in range(N,2*N-1):
    A[ii,ii] = -1/deta
    A[ii,ii+1] = 1/deta

# boundary conditions for u at etamax ~ infinity
A[2*N-1,2*N-1] = 1

# boundary condition for f(0)
A[2*N,2*N] = 1

# Blausius equation for f (derivative of f)
for ii in range(2*N+1, 3*N):
    A[ii,ii-1] = -1/deta
    A[ii,ii] = 1/deta

In [5]:
# now, a function returning vector b is defined

def b(y):
    
    result = np.zeros(3*N)
    
    # Blausius equation for xi (derivative of xi)
    for ii in range(N-1):
        result[ii] = y[ii]*y[2*N+ii]/2
        
    # Blausius equation for u (derivative of u)
    for ii in range(N,2*N-1):
        result[ii] = - y[ii-N]
        
    # boundary conditions for u at etamax ~ infinity
    result[2*N-1] = -1

    # Blausius equation for f (derivative of f)
    for ii in range(2*N+1, 3*N):
        result[ii] = -y[ii-1-N]
    
    return result

In [6]:
# Jacobi's matrix for b; this will come in handy later

def db(y):
    
    result = np.zeros((3*N,3*N))
    
    # Blausius equation for xi (derivative of xi)
    for ii in range(N-1):
        result[ii,2*N+ii] = y[ii]/2
        result[ii,ii] = y[2*N+ii]/2
        
    # Blausius equation for u (derivative of u)
    for ii in range(N,2*N-1):
        result[ii,ii-N] = - 1

    # Blausius equation for f (derivative of f)
    for ii in range(2*N+1, 3*N):
        result[ii,ii-1-N] = -1
        
    return result

We now have a non-linear equation in the form:

$$ F(y) = Ay + b(y) = 0$$

this can be solved with a Newton-Raphson scheme starting from an initial guess `y0`:

$$ y^{\,k+1} = y^{\,k} - J^{-1}(y^{\,k})\, F(y^{\,k}) $$

where $J$ is the Jacobi's matrix of $F$, $k$ denotes the iteration of the algorithm. Iterations are stopped when a maximum of iteratins `KMAX` is reached or the residual $f(y^{\,k})$ is below a threshold value `TOLL`.

In [7]:
# define F and the jacobian J
F = lambda yy : A.dot(yy) + b(yy)
J = lambda yy : A + db(yy)

In [ ]:
# initial condition (linear profile which respects B.C.)
y0 = np.zeros(3*N)
y0[0:N] = 0.5*linspace(1,0,n)*deta

In [11]:
KMAX = 100
TOLL = 1e-6

exit4it = True
for ii in range(KMAX):
    dy = - np.linalg.solve(J(y0),F(y0))
    y = y0 + dy
    y0 = y
    print(max(F(y)))
    if max(abs(dy)) < TOLL:
        exit4it = False
        print('Algorithm terminated due to satisfactory residual (number of iterations: {}).'.format(ii+1))
        break
if exit4it:
    print('Warning: algorithm was terminated since the iteration threshold has been reached;')
    print('converngence might have not been achieved.')

0.13122189440825743
2.4303756581120597
0.6417773081718352
0.20098413012794403
0.1411738211311502
0.42501390161825037
0.1527903196647448
0.25005579767287067
0.1296128350282217
25.895091317369406
6.506298086234909
1.6595941486310906
0.44999909083316775
0.1579318085116823
0.21893819357492242
0.1339122997638102
1.0048128628377944
0.28835174489719667
0.13081522079928876
3.1351589401983784
0.8175462005750362
0.24283756328572378
0.13001959157954796
7.428270325199257
1.8900042218349504
0.507243291225825
0.17026255042286556
0.1775789273560215
0.1638032527837365
0.1952654323613678
0.14483291600742873
0.3409213175424866
0.1374036225688713
0.5934808776770184
0.1897611022640279
0.14926630806429173
0.2810962341846074
0.1302627632973077
5.223509617119454
1.3390624686821502


KeyboardInterrupt: 

In [ ]:
print(y)